## Preface

You are encouraged to play with the code, run it on your own data, and exchange knowledge with other participants.

## Converting your data to OME-zarr

If you don't have your own data, you can use the `tibia.tiff` provided. The data is far from perfect, and relatively small (~250 MB) - to make this tutorial run fast. The data is of a CT scan of a mouse shin bone ("tibia"), and is courtesy of Mark Hopkinson (Royal Veterinary College).

In [1]:
import bioio
from pathlib import Path

In [2]:
# replace this with your own data
sample_data_path = Path("/media/alessandro/T7/tibiae/acc52_10um.tif")
assert sample_data_path.exists()
sample_data = bioio.BioImage(sample_data_path)
bioio.plugin_feasibility_report(sample_data_path)

{'bioio-tifffile': PluginSupport(supported=True, error=None),
 'bioio-sldy': PluginSupport(supported=False, error='Unable to find any images within the image directory'),
 'ArrayLike': PluginSupport(supported=False, error="ArrayLikeReader does not support the image: 'Unsupported image type: <class 'fsspec.implementations.local.LocalFileSystem'>. ArrayLikeReader supported types are numpy ndarray, dask Array,or xarray DataArray.'.")}

In [3]:
print(sample_data.metadata)
print(sample_data.dims)
print(sample_data.dims.order)
print(sample_data.channel_names)

ImageJ=1.54f
images=1672
slices=1672
unit=pixel
spacing=1.999401913875598
loop=false
<Dimensions [T: 1, C: 1, Z: 1672, Y: 349, X: 414]>
TCZYX
['Channel:0:0']


In [4]:
sample_data_dask_array = sample_data.dask_data
print(sample_data_dask_array.shape)
print(sample_data_dask_array.chunks)

(1, 1, 1672, 349, 414)
((1,), (1,), (1672,), (349,), (414,))


It may be helpful to have a system monitor (check particularly the RAM usage!) open!

In [5]:
sample_dask_array_rechunked = sample_data_dask_array.rechunk(chunks=(1,1,32,32,32))

not a lot of information here. Let convert to OME zarr to explore

reference in particular chapter 2 of textbook here

In [ ]:
from pydantic_zarr.v2 import ArraySpec
from numcodecs import Zstd

array_spec = ArraySpec(
    shape=sample_data_dask_array[0,0].shape,
    dtype=sample_data_dask_array.dtype,
    chunks=(32, 32, 32),
    compressor=Zstd(level=5),
)
print(array_spec)

ValidationError: 1 validation error for ArraySpec
  Value error, ('Length of shape must match length of chunks. Got 5 elements', 'for shape and 3 elements for chunks.') [type=value_error, input_value={'shape': (1, 1, 1672, 34...pressor': Zstd(level=5)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [7]:
from pathlib import Path
import tempfile
import zarr
from dask.array import core as dask_core

store = zarr.MemoryStore()
zarr_array = array_spec.to_zarr(store=store, path="/")

# arrays don't fit in memory so copy and transfer chunk by chunk
zarr_array[:] = sample_data.data[:]

In [12]:
from pathlib import Path
import tempfile
import os
import numpy as np

temp_dir = tempfile.TemporaryDirectory()
temp_path = Path(temp_dir.name)
print(f"Created temporary directory at {temp_path}")
print(f"Directory contents before saving: {os.listdir(temp_path)}")

store = zarr.DirectoryStore(temp_dir.name)
zarr_array = array_spec.to_zarr(store=store, path="/")
print(f"Directory contents after creating array: {os.listdir(temp_path)}")

rechunked = sample_data.dask_data.rechunk(chunks=(1,1,32,32,32))
rechunked = rechunked.squeeze()
zarr_array[:] = rechunked.compute()
print(f"Directory contents after adding data: {os.listdir(temp_path)}")
zarr_array

temp_dir.cleanup()

if Path(temp_path).exists():
    print(f"Directory contents after cleanup: {os.listdir(temp_path)}")
else:
    print("Directory removed during cleanup")
zarr_array

Created temporary directory at /tmp/tmpmmf3odut
Directory contents before saving: []
Directory contents after creating array: ['.zarray', '.zattrs']


ValueError: parameter 'value': expected array with shape (1, 1, 1672, 349, 414), got (1672, 349, 414)

In [9]:
zarr_array.chunks

(1, 1, 32, 32, 32)

In [ ]:
# import napari
# viewer = napari.Viewer()
# viewer.add_image(zarr_array)
# napari.run()

In [11]:
from ome_zarr_models.v04 import Image
from ome_zarr_models.v04.axes import Axis

voxel_size = 10
ome_zarr_image = Image.new(
    array_specs = [ArraySpec.from_array(zarr_array)],
    paths = ["level0"],
    axes = [
        Axis(name="x", type="space", unit="um"),
        Axis(name="y", type="space", unit="um"),
        Axis(name="z", type="space", unit="um")
    ],
    global_scale = [voxel_size, voxel_size, voxel_size],
    scales = [[1, 1, 1]],
    translations = [[0, 0, 0]],
    name = "bone image"
)
print(ome_zarr_image)

ValidationError: 1 validation error for Image
  Value error, The multiscale metadata has 3 axes which does not match the dimensionality of the array found in this group at level0 (5). The number of axes must match the array dimensionality. [type=value_error, input_value={'members': {'level0': Ar...on='0.4')], omero=None)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

Eva Maxfield Brown, Dan Toloudis, Jamie Sherman, Madison Swain-Bowden, Talley Lambert, Sean Meharry, Brian Whitney, AICSImageIO Contributors (2023). BioIO: Image Reading, Metadata Conversion, and Image Writing for Microscopy Images in Pure Python [Computer software]. GitHub. https://github.com/bioio-devs/bioio